# PDIoT Group A1 model

### Imports

In [1]:
import os
import numpy as np
import tensorflow as tf

from ml.config import cfg
from ml.dataset import odgt2data
from ml.models import ModelBuilder, OptimizerBuilder, LRScheduleBuilder, TimingCallback

print("All packages imported!")

2023-11-17 17:18:09.114220: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


All packages imported!


### Config

In [2]:
config_fp = '/home/jonathan/git/pdiot-ml/config/motion-mlp.yaml'
cfg.merge_from_file(config_fp)
cfg.DATASET.path = '/home/jonathan/git/pdiot-ml/data/sets/pdiot-data/'
cfg.TRAIN.path = '/home/jonathan/git/pdiot-ml/ckpt/motion-mlp/'   
print(cfg)

DATASET:
  LIST:
    test: test_motion_pdiot-data.odgt
    train: train_motion_pdiot-data.odgt
    val: val_motion_pdiot-data.odgt
  num_classes: 2
  path: /home/jonathan/git/pdiot-ml/data/sets/pdiot-data/
MODEL:
  ARCH:
    LSTM:
      hidden_size: 0
      num_layers: 0
    MLP:
      dropout: 0.2
      hidden_size: 128
      num_layers: 2
  INPUT:
    format: normal
    sensor: all
    window_size: 15
TRAIN:
  DATA:
    batch_size: 128
    overlap_size: 12
  FN:
    config: config.yaml
    history: history.csv
    log: log.txt
    weight: weights.hdf5
  LEN:
    early_stop: 10
    num_epoch: 100
  LR:
    gamma: 0.1
    schedule: step
    step_size: 40
  OPTIM:
    lr: 0.001
    momentum: 0.9
    optim: adam
    weight_decay: 0.0005
  path: /home/jonathan/git/pdiot-ml/ckpt/motion-mlp/


### Data

In [3]:
train_odgt_fp = os.path.join(cfg.DATASET.path, cfg.DATASET.LIST.train)
val_odgt_fp = os.path.join(cfg.DATASET.path, cfg.DATASET.LIST.val)

train_X, train_y = odgt2data(train_odgt_fp, cfg.MODEL.INPUT.window_size, 
                                            cfg.TRAIN.DATA.overlap_size, 
                                            cfg.DATASET.num_classes)
val_X, val_y = odgt2data(val_odgt_fp, cfg.MODEL.INPUT.window_size, 
                                      cfg.TRAIN.DATA.overlap_size,
                                      cfg.DATASET.num_classes)

### Model

In [4]:
model = ModelBuilder.build_classifier(cfg.MODEL, '', cfg.DATASET.num_classes)
optimizer = OptimizerBuilder.build_optimizer(cfg.TRAIN.OPTIM)
loss = tf.keras.losses.SparseCategoricalCrossentropy()
metrics = [
    tf.keras.metrics.SparseCategoricalAccuracy(name='acc'),
]

lr_scheduler = LRScheduleBuilder.build_scheduler(cfg.TRAIN.LR)
lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_scheduler)

early_stop_callback = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=cfg.TRAIN.LEN.early_stop)

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=os.path.join(cfg.TRAIN.path, 'weights.hdf5'),
    save_weights_only=True,
    monitor='val_acc',
    mode='max',
    save_best_only=True)

timing_callback = TimingCallback()

history_callback = tf.keras.callbacks.CSVLogger(
    os.path.join(cfg.TRAIN.path, 'history.csv'), 
    separator=',', 
    append=False)

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
model.fit(train_X, train_y, validation_data=(val_X, val_y), 
          epochs=cfg.TRAIN.LEN.num_epoch, 
          batch_size=cfg.TRAIN.DATA.batch_size,
          callbacks=[lr_callback, 
                     early_stop_callback,
                     checkpoint_callback,
                     timing_callback,
                     history_callback])

2023-11-17 17:18:17.091060: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Epoch 1/100
5709/5709 [==============================] - 47s 8ms/step - loss: 0.1190 - acc: 0.9543 - val_loss: 0.0733 - val_acc: 0.9738 - lr: 0.0010 - time: 46.5467
Epoch 2/100
5709/5709 [==============================] - 54s 9ms/step - loss: 0.0780 - acc: 0.9690 - val_loss: 0.0714 - val_acc: 0.9724 - lr: 0.0010 - time: 53.6730
Epoch 3/100
5709/5709 [==============================] - 64s 11ms/step - loss: 0.0718 - acc: 0.9721 - val_loss: 0.0660 - val_acc: 0.9774 - lr: 0.0010 - time: 63.8896
Epoch 4/100
5709/5709 [==============================] - 71s 12ms/step - loss: 0.0687 - acc: 0.9735 - val_loss: 0.0668 - val_acc: 0.9739 - lr: 0.0010 - time: 71.1809
Epoch 5/100
5709/5709 [==============================] - 67s 12ms/step - loss: 0.0663 - acc: 0.9746 - val_loss: 0.0621 - val_acc: 0.9769 - lr: 0.0010 - time: 66.6827
Epoch 6/100
5709/5709 [==============================] - 68s 12ms/step - loss: 0.0645 - acc: 0.9755 - val_loss: 0.0585 - val_acc: 0.9785 - lr: 0.0010 - time: 67.6070
Epoch 

KeyboardInterrupt: 

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_1 (Sequential)   (None, 150)               0         
                                                                 
 dense (Dense)               (None, 128)               19328     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 5)                 645       
                                                                 
Total params: 36,485
Trainable params: 36,485
Non-traina